In [2]:
seq = ''
with open('ecoli_k12_genome.fasta') as f:
    f.readline() # discard header
    for line in f.readlines():
        seq += line.strip()
        
print('This genome is {} basepairs long'.format(len(seq)))

This genome is 4639221 basepairs long


In [17]:
import numpy as np
import subprocess
import requests
import os
import time
from IPython.core.display import display, Image 
import uuid
import glob

api_url = 'http://140.221.67.148:8000'
#api_url = 'http://kbase.us/services/assembly'

# play with gc content
def calc_gc(s):
    gc = 0
    for c in s:
        if c == 'g' or c == 'c':
            gc += 1
    return float(gc) / len(s)
    
def generate_random_genome(genome_size):
    from numpy.random.mtrand import RandomState
    rand = RandomState()
    return rand.choice(['A','T','C','G'], genome_size).tostring()

def sim_assembly(genome, out_prefix, assembler_str='spades', insert=180, stdev=0, readlen=100):
    dwg_cmd = 'dwgsim/dwgsim -e 0 -E 0.0 -1 {0} -2 {0} -d {1} -s {2} -Q 0 -r 0.0 -y 0.0 {3} {4}'.format(readlen, insert,
                                                                                                        stdev, genome, out_prefix)
    print dwg_cmd
    p = subprocess.Popen(dwg_cmd.split())
    p.wait()
    arast_cmd = 'arast -s {} run --curl --pair {} {} --reference {} -a {} -m "err_classification"'.format(
        api_url,
        '{}.bwa.read1.fastq'.format(out_prefix),
                                    '{}.bwa.read2.fastq'.format(out_prefix), 
                                    genome, assembler_str)
    print arast_cmd
    return subprocess.check_output(arast_cmd, shell=True).strip(), genome


def parse_job_report(r):
    return '\n'.join(r.split('\n')[:50])

def get_job_report(job_id):
    while True:
        status = requests.get('{}/user/cbun/job/{}/status'.format(api_url, job_id)).text
        if status == 'Complete':
            break
        time.sleep(5)
    requests.get('{}/static/serve/cbun/job/{}'.format(api_url, job_id))
    return requests.get('{0}/static/cbun/job/{1}/{1}_report.txt'.format(api_url, job_id)).text



def nucmer_filter(ref, asm, prefix, min_align_len=9000, outdir=None, min_len=50):
    if not outdir:
        outdir = os.getcwd()
    cmd = 'nucmer {} {} -p {} -l {} --maxmatch'.format(ref, asm, prefix, min_len)    
    p = subprocess.Popen(cmd, shell=True, cwd=outdir)
    print cmd
    p.wait()
    delta_cmd = 'delta-filter -l {0} {1}.delta {1}.filtered.delta'.format(min_align_len, prefix)
    print delta_cmd
    p = subprocess.Popen(delta_cmd, shell=True, cwd=outdir)
    p.wait()
    
    m_cmd = 'mummerplot {0}.filtered.delta --png -p {0}'.format(prefix)
    print m_cmd
    p = subprocess.Popen(m_cmd, shell=True, cwd=outdir)
    p.wait()
    
    return '{}.png'.format(prefix), '{}.filtered.delta'.format(prefix)

def split_contigs(input_handle, output_handle, sub_contig_len=10000):
    from Bio import SeqIO
    sub_contigs = []

    for record in SeqIO.parse(input_handle, "fasta"):
        if len(record.seq) > sub_contig_len:
            pos = 0
            counter = 0
            while pos < len(record.seq):
                sub_contig = record[pos:pos+sub_contig_len]
                sub_contig.id += '.{}'.format(counter)
                #print sub_contig.id
                sub_contigs.append(sub_contig)
                pos += sub_contig_len
                counter += 1

    SeqIO.write(sub_contigs, output_handle, "fasta")



def analyze_job(job_id, genome, min_aln_len=20):
    report = get_job_report(job_id)
    wd = os.path.join(os.getcwd(), '{}_{}'.format(job_id, uuid.uuid4()))
    contig_dir = os.path.join(wd, 'contigs')
    os.makedirs(contig_dir)
    cmd = 'arast -s {} get -j {} -a -o {}'.format(api_url, job_id, contig_dir)
    subprocess.check_output(cmd, shell=True)
    print subprocess.check_output(cmd, shell=True)
    pngs = []
    for f in os.listdir(contig_dir):
        contig = os.path.join(contig_dir, f)
        split_fa = os.path.join(contig_dir, 'split.{}'.format(f))
        split_contigs(contig, split_fa)
        nucmer_filter(genome, split_fa, split_fa)
        pngs = glob.glob(wd + '/*.png')
    return report, pngs

In [8]:
g = './ecoli_k12_genome.fasta'
sim_assembly(g, 'sim_e', assembler_str='spades', insert=180, stdev=0, readlen=100)

dwgsim/dwgsim -e 0 -E 0.0 -1 100 -2 100 -d 180 -s 0 -Q 0 -r 0.0 -y 0.0 ./ecoli_k12_genome.fasta sim_e
arast -s http://140.221.67.148:8000 run --curl --pair sim_e.bwa.read1.fastq sim_e.bwa.read2.fastq --reference ./ecoli_k12_genome.fasta -a spades -m "err_classification"


('Job ID: 1220', './ecoli_k12_genome.fasta')

In [18]:
analyze_job(1220, './ecoli_k12_genome.fasta')


nucmer ./ecoli_k12_genome.fasta /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta -p /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta -l 50 --maxmatch
delta-filter -l 9000 /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta.delta /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta.filtered.delta
mummerplot /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta.filtered.delta --png -p /Users/cbun/Development/ipython/1220_e3fd4938-9310-4674-ac0e-504f9508958c/contigs/split.1220_1.spades_contigs.fasta


(u'<!DOCTYPE html PUBLIC\n"-//W3C//DTD XHTML 1.0 Transitional//EN"\n"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html>\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"></meta>\n    <title>404 Not Found</title>\n    <style type="text/css">\n    #powered_by {\n        margin-top: 20px;\n        border-top: 2px solid black;\n        font-style: italic;\n    }\n\n    #traceback {\n        color: red;\n    }\n    </style>\n</head>\n    <body>\n        <h2>404 Not Found</h2>\n        <p>Nothing matches the given URI</p>\n        <pre id="traceback"></pre>\n    <div id="powered_by">\n      <span>\n        Powered by <a href="http://www.cherrypy.org">CherryPy 3.7.0</a>\n      </span>\n    </div>\n    </body>\n</html>\n',
 [])